## Introduction

In this tutorial, we’ll walk through the process of interacting with a Google Cloud Storage (GCS) bucket named dauphine-bucket, specifically focusing on the data directory within the bucket. We’ll cover how to:

- List all files in the bucket’s data directory.
- Retrieve information about a specific file.
- Read files using the Unstructured library.
- Visualize the extracted documents with LangChain.

This guide is intended for users who are familiar with Python and basic cloud storage concepts.

Prerequisites

Before we begin, ensure you have the following:

- Python 3.x installed on your system.
- Access to the GCP bucket dauphine-bucket/data with the necessary permissions.
- Google Cloud SDK installed and authenticated. You can authenticate by running:

In [3]:
!gcloud auth login

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=hlauMlQRVUnpLHTOrTkNwuXr25vzDE&access_type=offline&code_challenge=wqVIThC9LNDNA1RG-_gz6sfJcDLmA4nspUrnCWfHOfs&code_challenge_method=S256


You are now logged in as [elyesmaalel@gmail.com].
Your current project is [dauphine-437611].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID


The following Python libraries installed:
- google-cloud-storage
- unstructured
- langchain

To know more about the libraries, you can visit the following links:
- [google-cloud-storage](https://googleapis.dev/python/storage/latest/index.html)
- [unstructured](https://docs.unstructured.io/examplecode/codesamples/oss/vector-database)
- [langchain](https://langchain.readthedocs.io/en/latest/)


In [5]:
%pip install -q google-cloud-storage unstructured langchain python-magic sqlalchemy langchain_google_cloud_sql_pg
%pip install -q "unstructured[pptx]"

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# I. Listing and loading files from a GCS bucket

### I.1. Listing Files in the GCP Bucket

Explanation:

To interact with a GCS bucket, we’ll use the google-cloud-storage library. We’ll initialize a client, access the bucket, and list all the files within the data directory.

Code:

In [4]:
!gcloud auth application-default login


Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=A7diTGYG6YDkdWlSBPt2HFE0Xiqsj0&access_type=offline&code_challenge=4fbFJvnUyj74FIsoi6BpmM5KrcQfr7CA6wyTgp6CGl0&code_challenge_method=S256


Credentials saved to file: [C:\Users\Elyessss\AppData\Roaming\gcloud\application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).

Quota project "dauphine-437611" was added to ADC which can be used by Google client libraries for billing and quota. Note that some services may still bill the project owning the resource.


In [6]:
# Import the necessary library
from google.cloud import storage

# Initialize a client
client = storage.Client()

# Access the bucket
bucket_name = 'dauphine-bucket'
bucket = client.get_bucket(bucket_name)

# List all files in the 'data' directory
blobs = bucket.list_blobs(prefix="data/")

print("Files in 'dauphine-bucket/data':")
for blob in blobs:
    print(blob.name)

Files in 'dauphine-bucket/data':
data/
data/1 - Gen AI - Dauphine Tunis.pptx
data/2.1 - Before Transformers - Gen AI - Dauphine Tunis.pptx
data/2.2  - Transformers - Gen AI - Dauphine Tunis.pptx
data/3 - Retrieval Augmented Generation - Gen AI - Dauphine Tunis.pptx


Output Explanation:

Running this code will display all the file paths within the data directory of the bucket. The prefix='data/' parameter ensures we only get files from that specific directory.

### I.2. Getting Information About One File


Explanation:

Sometimes, you may need detailed information about a specific file, such as its size, content type, or the last time it was updated. We’ll retrieve this metadata for a chosen file.


In [7]:
# Specify the file path (replace with an actual file from your bucket)
file_path = 'data/1 - Gen AI - Dauphine Tunis.pptx'

# Get the blob object
blob = bucket.get_blob(file_path)

# TODO
if blob:
    print(f"Information for '{file_path}':")
    print(f"Size: {blob.size} bytes")
    print(f"Content Type: {blob.content_type}")
    print(f"Updated On: # TODO")
    print(f"Blob name: {blob.name}")
else:
    print(f"File '{file_path}' not found in the bucket.")

Information for 'data/1 - Gen AI - Dauphine Tunis.pptx':
Size: 6724048 bytes
Content Type: application/vnd.openxmlformats-officedocument.presentationml.presentation
Updated On: # TODO
Blob name: data/1 - Gen AI - Dauphine Tunis.pptx


Output Explanation:

This code will output metadata about the specified file. Make sure to replace 'data/your_file.ext' with the actual file path.

### I.3. Reading Files with Unstructured

Explanation:

The Unstructured library allows us to parse and process unstructured data from various file formats. We’ll download a file from the bucket and use Unstructured to read and extract its content.

In [ ]:
!pip install langchain-unstructured


In [8]:
# Import necessary libraries
from langchain_core.documents import Document
import os
from google.cloud.storage.bucket import Bucket
from langchain_unstructured import UnstructuredLoader


DOWNLOADED_LOCAL_DIRECTORY = r"C:\Users\Elyessss\OneDrive\Bureau\cloudd\GenAI-GCP\exercices\tp_4\downloaded_files"


# Function to download the file: file_path from the GCS Bucket
def download_file_from_bucket(bucket: Bucket, file_path: str) -> str:
    # Download the file locally
    blob = bucket.blob(file_path)

    # data/1 - Gen AI - Dauphine Tunis.pptx' -> 1 - Gen AI - Dauphine Tunis.pptx'
    local_file_name = os.path.basename(file_path)

    # ./downloaded_files/1 - Gen AI - Dauphine Tunis.pptx
    local_filepath = os.path.join(DOWNLOADED_LOCAL_DIRECTORY, local_file_name)

    blob.download_to_filename(local_filepath)
    print(f"Downloaded '{file_path}' to '{local_file_name}'")
    return local_filepath


def read_file_from_local(local_filepath: str) -> list[Document]:
    # Use Unstructured to read the file
    loader = UnstructuredLoader(local_filepath)
    documents = loader.lazy_load()
    return documents

In [9]:
# Load all the
blobs = list(bucket.list_blobs(prefix='data/'))
documents: list[Document] = []
if blobs:
    for blob in blobs:
        try:
            local_filepath = download_file_from_bucket(bucket, blob.name)
            documents.extend(read_file_from_local(local_filepath))
        except Exception as e:
            print(f"An error occurred while processing '{blob.name}': {e}")
else:
    print("No files found in the 'data' directory.")

An error occurred while processing 'data/': [Errno 2] No such file or directory: 'C:\\Users\\Elyessss\\OneDrive\\Bureau\\cloudd\\GenAI-GCP\\exercices\\tp_4\\downloaded_files\\'
Downloaded 'data/1 - Gen AI - Dauphine Tunis.pptx' to '1 - Gen AI - Dauphine Tunis.pptx'
Downloaded 'data/2.1 - Before Transformers - Gen AI - Dauphine Tunis.pptx' to '2.1 - Before Transformers - Gen AI - Dauphine Tunis.pptx'
Downloaded 'data/2.2  - Transformers - Gen AI - Dauphine Tunis.pptx' to '2.2  - Transformers - Gen AI - Dauphine Tunis.pptx'
Downloaded 'data/3 - Retrieval Augmented Generation - Gen AI - Dauphine Tunis.pptx' to '3 - Retrieval Augmented Generation - Gen AI - Dauphine Tunis.pptx'


### I.4. Visualizing the First Documents Extracted with LangChain

Explanation:

LangChain is a framework for developing applications powered by language models. We’ll use it to load and visualize the documents extracted from the file.

In [10]:
for doc in documents[:3]:
    print(f"Content:\n{doc.page_content}\nMetadata:\n{doc.metadata}\n")

Content:
Generative AI with LLM
Metadata:
{'source': 'C:\\Users\\Elyessss\\OneDrive\\Bureau\\cloudd\\GenAI-GCP\\exercices\\tp_4\\downloaded_files\\1 - Gen AI - Dauphine Tunis.pptx', 'category_depth': 1, 'file_directory': 'C:\\Users\\Elyessss\\OneDrive\\Bureau\\cloudd\\GenAI-GCP\\exercices\\tp_4\\downloaded_files', 'filename': '1 - Gen AI - Dauphine Tunis.pptx', 'last_modified': '2024-12-11T15:34:23', 'page_number': 1, 'languages': ['eng'], 'filetype': 'application/vnd.openxmlformats-officedocument.presentationml.presentation', 'category': 'Title', 'element_id': '48f84e1bdca7ae95acb10169474c3135'}

Content:
Florian Bastin
Metadata:
{'source': 'C:\\Users\\Elyessss\\OneDrive\\Bureau\\cloudd\\GenAI-GCP\\exercices\\tp_4\\downloaded_files\\1 - Gen AI - Dauphine Tunis.pptx', 'category_depth': 1, 'file_directory': 'C:\\Users\\Elyessss\\OneDrive\\Bureau\\cloudd\\GenAI-GCP\\exercices\\tp_4\\downloaded_files', 'filename': '1 - Gen AI - Dauphine Tunis.pptx', 'last_modified': '2024-12-11T15:34:23',

### I.5. Join extracted document by page

Explanation:

- The text extraction block is uninformative because very small text blocks are extracted from the document.
- We can join the extracted text by page to get a more meaningful output.
- A metadata with the 'page_number' can be helpful
- The other metadatas need to be merged

In [11]:
from collections import defaultdict


# Function to merge documents by page number
def merge_documents_by_page(documents: list[Document]) -> list[Document]:
    merged_documents: list[Document] = []
    page_dict = {}

    # Group documents by page number
    for doc in documents:
        document_source = doc.metadata.get("source")
        page_number = doc.metadata.get("page_number")

        if page_number is not None and document_source is not None:
            key = (document_source, page_number)
            if key not in page_dict:
                page_dict[key] = [doc]
            else:
                page_dict[key].append(doc)

    # Merge documents for each page
    for (document_source, page_number), docs in page_dict.items():
        if docs:
            # Use the metadata of the first document in the group
            merged_metadata = docs[0].metadata
            # Concatenate the page content of all documents in the group
            merged_content = "\n".join(doc.page_content for doc in docs)
            # Create a new Document with merged content and metadata
            merged_documents.append(
                Document(metadata=merged_metadata, page_content=merged_content)
            )

    return merged_documents


# Merge the documents by page
merged_documents = merge_documents_by_page(documents)

# Print the merged documents
for doc in merged_documents:
    print("-" * 50)
    print(f"\nPage Number: {doc.metadata.get('page_number')}")
    print(f"Content:\n{doc.page_content}\nMetadata:\n{doc.metadata}\n")
    print("-" * 50)

--------------------------------------------------

Page Number: 1
Content:
Generative AI with LLM
Florian Bastin
👨🏼‍🎓 Master MASH - Université PSL
👨🏼‍💻 LLM Engineer @OctoTechnology
Le Monde, Casino, Channel, Club Med, Pernod Ricard, Suez
‹#›
Metadata:
{'source': 'C:\\Users\\Elyessss\\OneDrive\\Bureau\\cloudd\\GenAI-GCP\\exercices\\tp_4\\downloaded_files\\1 - Gen AI - Dauphine Tunis.pptx', 'category_depth': 1, 'file_directory': 'C:\\Users\\Elyessss\\OneDrive\\Bureau\\cloudd\\GenAI-GCP\\exercices\\tp_4\\downloaded_files', 'filename': '1 - Gen AI - Dauphine Tunis.pptx', 'last_modified': '2024-12-11T15:34:23', 'page_number': 1, 'languages': ['eng'], 'filetype': 'application/vnd.openxmlformats-officedocument.presentationml.presentation', 'category': 'Title', 'element_id': '48f84e1bdca7ae95acb10169474c3135'}

--------------------------------------------------
--------------------------------------------------

Page Number: 2
Content:
I.A Pretraining Large Language Model
A. Pretraining a Lar

# II. Ingesting in Cloud SQL

We will ingest each merged_document in Cloud SQL.

ALREADY DONE by teacher: 
- Create a Cloud SQL instance
- Create a database in the instance


TODO:
- Create a table in CloudSQL with you initials
- Create the schema of the table
- Ingest the data in the table


Follow this [documentation](https://python.langchain.com/docs/integrations/vectorstores/google_cloud_sql_pg/)

### II.1 Understand how to connect to Cloud SQL 


First we need to connect to Cloud SQL 
- Follow this [link](https://cloud.google.com/sql/docs/postgres/connect-instance-auth-proxy) to understand how it works

Then be familiar ith the following PostgreSQL commands:
```bash 
`psql "host=127.0.0.1 port=5432 sslmode=disable dbname=gen_ai_db user=postgres"` # to connect to the user `postgres`
# the user we use is `students`
# a password provided by the teacher is required
`\l` # to list all databases
`\c gen_ai_db` # to connect to the database `gen_ai_db`
`\dt` # to list all tables
`\d+ table_name` # to describe a table
`SELECT * FROM table_name` # to select all rows from a table
`\du` # to list all users
`\q` # to quit
`CREATE DATABASE db_name;` # to create a database
`CREATE USER user_name WITH PASSWORD 'password';` # to create a user
`GRANT ALL PRIVILEGES ON DATABASE db_name TO user_name;` # to grant all privileges to a user on a database
`GRANT ALL PRIVILEGES ON ALL TABLES IN SCHEMA public TO user_name;` # to grant all privileges to a user on all tables in a schema
`ALTER USER user_name WITH SUPERUSER;` # to grant superuser privileges to a user
`DROP DATABASE db_name;` # to drop a database
`DROP USER user_name;` # to drop a user
`DROP TABLE table_name;` # to drop a table
`REVOKE ALL PRIVILEGES ON DATABASE db_name FROM user_name;` # to revoke all privileges from a user on a database
```

When Cloud SQL Proxy is downloaded and the tutorial is followed. You should be connected to the instance. 
You can connect to the dabase as a user `students` with the password provided by the teacher.
  - `psql "host=127.0.0.1 port=5432 sslmode=disable dbname=gen_ai_db user=students"`
  - Enter the password provided by the teacher
Try to create a table `initial_tests_table` with the following schema:
  - `CREATE TABLE initial_tests_table (id SERIAL PRIMARY KEY, document TEXT, page_number INT, title TEXT, author TEXT, date TEXT);`
  - `\dt` to check if the table has been created
  - `\d+ initial_tests_table` to check the schema of the table
  - `DROP TABLE initial_tests_table;` to drop the table
  - `\q` to quit


In [12]:
%pip install --upgrade --quiet  langchain-google-cloud-sql-pg langchain-google-vertexai

Note: you may need to restart the kernel to use updated packages.


In [13]:
from dotenv import load_dotenv
load_dotenv()

False

In [19]:
import os
from config import PROJECT_ID, REGION, INSTANCE, DATABASE, DB_USER

DB_PASSWORD = os.environ["DB_PASSWORD"]

In [20]:
TABLE_NAME = "em_table" # Table name in the database initials-table. Ex: fb_table

In [21]:
from langchain_google_cloud_sql_pg import PostgresEngine

# Connect to the PostgreSQL database
engine = PostgresEngine.from_instance(
    project_id=PROJECT_ID,
    instance=INSTANCE,
    region=REGION,
    database=DATABASE,
    user=DB_USER,
    password=DB_PASSWORD,
)


In [22]:
# Create a table in the PostgreSQL database with the required columns
from sqlalchemy.exc import ProgrammingError

try:
    await engine.ainit_vectorstore_table(
        table_name=TABLE_NAME, # Vector size for VertexAI model(textembedding-gecko@latest)
        vector_size=768,
    )
except ProgrammingError:
    print("Table already created")

- Execute \d+ [YOUR_INITIALS]_table in the psql shell to check the schema of the table

### II.2 Create an embedding to convert your documents

In [23]:
from langchain_google_vertexai import VertexAIEmbeddings

embedding = VertexAIEmbeddings(
    model_name="textembedding-gecko@latest", project=PROJECT_ID
)

In [24]:
from langchain_google_cloud_sql_pg import PostgresVectorStore

vector_store = PostgresVectorStore.create_sync(  # Use .create() to initialize an async vector store
    engine=engine,
    table_name=TABLE_NAME,
    embedding_service=embedding,
)

In [25]:
 vector_store.add_documents(merged_documents)
# Excute only once this cell

['89018f22-60d7-408a-ae0e-8c67bfae3110',
 '667c1327-b037-4f00-af69-5abc503b5b4f',
 'a99a4acc-fb04-41d3-8eb4-5067f056a01b',
 '15d96f16-d6a3-4d40-958f-8215c9edb5f3',
 '5a4e068c-5f5c-4262-9452-cfae5102f526',
 '2030228f-f3f7-47ed-8585-f53b60a0e519',
 '4027be6b-ed37-4df9-9505-a3a5c3f2d877',
 'e6d974ef-e5fe-46c3-b97f-d63936f63994',
 'cdfd0762-30bb-46eb-b105-7dd03373d271',
 '9fe4b469-6e4d-44f4-8eb5-576dc34fcd99',
 '09709ac7-f501-4025-baaf-c2edccc053f4',
 '0433b5dc-1329-4643-8f93-3adb042fe01a',
 'ee679f8f-8c91-44fc-a8ea-a2447c05bd86',
 '15136fc1-331c-4e23-bbb8-5ff502edefe5',
 '3df544a5-549a-4497-b515-cead82abaad4',
 '6fb07773-f240-4900-b1ba-d032521f18a0',
 '3e57ee35-6429-4d16-9198-6026ed5c3a2d',
 '96946768-41f3-4bd2-a2ee-1479e5f77974',
 'ea1cd9a5-6005-4306-aeac-2f9963696f26',
 'f3126676-932a-401d-b490-70536de1c46e',
 'ee831b22-1077-4906-b418-d19c0858540f',
 'a6a7bd1a-b2fe-4bd1-a721-bd8c2787c143',
 'b091d1ef-8315-46fc-8161-112a14b0bd7d',
 'd9f7f15c-9252-4281-a915-47e71bc0df03',
 '547a8e43-467f-

### II.3 Perform a similarity search

In [26]:
query = "How to train a Large Language Model?"

In [27]:
retriever = vector_store.as_retriever(
    search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.5}
)

docs = retriever.invoke(query)

In [28]:
for doc in docs:
    print("-" * 50)
    print("Content: ", doc.page_content)
    print("Metadata: ", doc.metadata)

--------------------------------------------------
Content:  I.A.7 Training Process
Training process
Steps 
Find scaling recipes (example: learning rate decrease if the size of the model increase)
Tune hyper parameters on small models of differents size
Choose the best models among the smallest ones
Train the biggest model with the 
Q. Should I use Transformers or LSTM ? 
‹#›
Stanford CS229 I Machine Learning I Building Large Language Models (LLMs) [Youtube]
Metadata:  {'source': 'C:\\Users\\Elyessss\\OneDrive\\Bureau\\cloudd\\GenAI-GCP\\exercices\\tp_4\\downloaded_files\\1 - Gen AI - Dauphine Tunis.pptx', 'category_depth': 0, 'file_directory': 'C:\\Users\\Elyessss\\OneDrive\\Bureau\\cloudd\\GenAI-GCP\\exercices\\tp_4\\downloaded_files', 'filename': '1 - Gen AI - Dauphine Tunis.pptx', 'last_modified': '2024-12-11T15:34:23', 'page_number': 14, 'languages': ['eng'], 'filetype': 'application/vnd.openxmlformats-officedocument.presentationml.presentation', 'category': 'Title', 'element_id':

**Congratulations**! You have successfully ingested the data in Cloud SQL.